










































**Install Required Packages**

In [ ]:
!pip install pyngrok
!pip install streamlit langchain langchain_google_genai python-dotenv pyngrok
!pip install langchain
!pip install langchain_openai langchain_community langchain pymysql pandas sentence-transformers langchain_experimental -q
!pip install python-dotenv
!pip install langchain_anthropic -q
!pip install langchain-google-genai -q
!pip install chromadb
!pip install --upgrade --quiet  langchain-google-genai
!pip install sqlalchemy
!pip install streamlit -q
!pip install --upgrade pandasai

In [ ]:
%%writefile app.py
#Import Packages.
import streamlit as st
import sqlite3
import os
import pandasai
from PIL import Image
import pandas as pd
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from urllib.parse import quote_plus
from langchain_core.prompts import PromptTemplate, FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from pprint import pprint
from pandasai import SmartDataframe
from pandasai import Agent

# Load the environment variables
load_dotenv()
os.environ['GOOGLE_API_KEY'] = "ENTER YOUR GOOGLE API KEY HERE"
os.environ['PANDASAI_API_KEY'] = "ENTER YOUR PANDASAI API KEY HERE"

# Setup the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Prompt Template for SQL Query
sql_prompt_template = PromptTemplate.from_template(
    """You are an SQL expert. Given the following schema information and user question, generate an accurate SQL query.

    Schema Information:
    {schema_info}

    User Question:
    {question}

    Instructions:
    - Generate a valid SQL query that answers the user question based on the provided schema.
    - Ensure that table names and column names in the query match those in the schema.
    - The SQL query should be free from syntax errors and unnecessary whitespace.
    - The SQL query should be free of any comments or suggestions or Assumptions. For example, "-- Assuming that..." or "**Let's assume that...",etc..

    SQL Query: """
)

#Prompt Template for Answer
answer_prompt = PromptTemplate.from_template(
    """You are an expert in translating SQL results into clear and concise answers. Based on the following details, generate a natural language response to answer the user's question.

    User Question: {question}
    SQL Query: {query}
    SQL Result: {result}

    Instructions:
    - Summarize the SQL result in a way that directly answers the user's question.
    - Present the information clearly and in a user-friendly format.
    - If applicable, include units of measurement or contextual details.
    - Avoid unnecessary details or technical jargon.

    Answer: """
)

#Few Shot Examples
examples = [
    {
        "input": "I want to know, how many different company name there are along with the number of employees working in it and the company name.",
        "query": "SELECT SUBSTR(Email, INSTR(Email, '@') + 1, INSTR(SUBSTR(Email, INSTR(Email, '@') + 1), '.') - 1) AS company_name, COUNT(*) AS employee_count FROM account GROUP BY company_name;"
    },
    {
        "input": "Total people who attended the event 'NASSCOM Design & Engineering Summit'",
        "query": "SELECT COUNT(DISTINCT account_id) AS total_attendees FROM (SELECT bo.account_id FROM booth_owner bo JOIN event e ON bo.event_id = e.event_id WHERE e.title = 'NASSCOM Design & Engineering Summit' UNION SELECT s.account_id FROM speaker s JOIN event e ON s.event_id = e.event_id WHERE e.title = 'NASSCOM Design & Engineering Summit') AS unique_attendees"
    }
    {
      "input": "List all the events along with their location and the number of attendees for each event.",
      "query": "SELECT e.title AS event_title, el.location_name AS event_location, COUNT(DISTINCT account_id) AS attendee_count FROM event e JOIN event_location el ON e.event_id = el.event_id LEFT JOIN booth_owner bo ON e.event_id = bo.event_id LEFT JOIN speaker s ON e.event_id = s.event_id GROUP BY e.title, el.location_name;"
    },
    {
      "input": "Find all segments in which the speaker 'John Doe' is presenting.",
      "query": "SELECT seg.segment_name FROM segment seg JOIN segment_speaker ss ON seg.segment_id = ss.segment_id JOIN speaker sp ON ss.speaker_id = sp.speaker_id WHERE sp.speaker_name = 'John Doe';"
    },
    {
      "input": "How many unique booths are there in the 'Tech Expo' event?",
      "query": "SELECT COUNT(DISTINCT bo.booth_id) AS unique_booth_count FROM booth bo JOIN event e ON bo.event_id = e.event_id WHERE e.title = 'Tech Expo';"
    },
    {
      "input": "List all the speakers and their corresponding segments for the event 'AI Conference'.",
      "query": "SELECT sp.speaker_name, seg.segment_name FROM speaker sp JOIN segment_speaker ss ON sp.speaker_id = ss.speaker_id JOIN segment seg ON ss.segment_id = seg.segment_id JOIN event e ON seg.event_id = e.event_id WHERE e.title = 'AI Conference';"
    },
    {
      "input": "Total number of events held in the 'New York' location.",
      "query": "SELECT COUNT(DISTINCT e.event_id) AS total_events FROM event e JOIN event_location el ON e.event_id = el.event_id WHERE el.location_name = 'New York';"
  }
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}\nSQL Query:"), ("ai", "{query}")]
)

# Defining the few-shot prompt
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["input"]
)

# Define the final prompt
final_prompt = ChatPromptTemplate.from_messages(
    [few_shot_prompt, ('human', '{input}')]
)

#Function for Schema Information Extraction of the uploaded database
def extract_schema_from_db(db_path):
    schema_info = {}
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    #Extract Tables Names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]
        #Extract Column names and their Data Types
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_info = {col[1]: col[2] for col in columns}
        schema_info[table_name] = column_info

    conn.close()
    return schema_info, [table[0] for table in tables]

#Setup the Streamlit App
st.title("SQL Query Generation and Execution")

#Upload the database file
db_file = st.file_uploader("Upload your Database.", type=["db", "sqlite"])

#Database is Connected
if db_file:
    with open("temp_db.sqlite", "wb") as f:
        f.write(db_file.getbuffer())

    if st.button("Connect"):
        #Connect Button for Database Connection
        try:
            db = SQLDatabase.from_uri("sqlite:///temp_db.sqlite")
            st.session_state["db"] = db
            st.session_state["db_connected"] = True

            #Schema Extraction of the Database
            schema_info, table_names = extract_schema_from_db("temp_db.sqlite")
            st.session_state["schema_info"] = schema_info
            st.session_state["table_names"] = table_names

            #Display Sucessful Connection Message and table Names
            st.success("Database connection successful!")
            st.write("Tables available in the Database:")
            for table in table_names:
                st.write(f"- {table}")

        except Exception as e:
            st.error(f"Database Connection Unsuccessful: {e}")

def UserInputQuery():
    #Input question for SQL generation
    if "db_connected" in st.session_state and st.session_state["db_connected"]:
        question = st.text_input("Ask a question about the Event:", "")

        if question:
            db = st.session_state.get("db")
            schema_info = st.session_state.get("schema_info")
            query_chain = sql_prompt_template | llm | StrOutputParser()
            execute_query = QuerySQLDataBaseTool(db=db)
            rephrase_answer = answer_prompt | llm | StrOutputParser()

            #Remove Additional Symbol or String from the query.
            def validate_sql_query(query):
                query = query.replace("sql", "")
                query = query.replace("`", "")
                query_lines = query.strip().split("\n")
                sql_query_lines = [line for line in query_lines if not line.strip().startswith("--")]
                sql_query = "\n".join(sql_query_lines)
                return sql_query

            #Function to Generate SQL Query.
            def generate_and_execute_query(question):
                sql_query = query_chain.invoke({"question": question, "schema_info": schema_info}) #Query is generated
                sql_query = validate_sql_query(sql_query)

                #Dispaly Generated SQL Query
                st.write("SQL Query generated based on the question:")
                st.code(sql_query)

                #Execute the Sql Query on Database
                if st.button("Execute the SQL Query"):
                    result = execute_query.run(sql_query) #Query is Executed

                    #Display the result from execution of SQl Command
                    st.write("Result for the generated SQL Query:")
                    data = pd.DataFrame(eval(result)) #Converting Data into Data Frame for compatibility with Agent
                    st.dataframe(data)  #Dispaly the Dataframe.

                    st.session_state['sql_query'] = sql_query
                    st.session_state['result'] = result

                    st.session_state.pop('visualization_image', None)

                    #Convert Data Column Names to String
                    data.columns = data.columns.astype(str)
                    pagent = Agent(data)

                    #Generate Visual Data
                    pagent_response = pagent.chat('Visualise the Dataframe with charts or graphs.')

                    # Display Visual Data
                    st.write("Data Visualisation:")

                    if isinstance(pagent_response, str) and pagent_response.endswith(".png"):
                      # Load and display the image if pagent_response is a path to an image
                      st.session_state['visualization_image'] = pagent_response
                      image = Image.open(pagent_response)
                      st.image(image, caption='Generated Data Visualisation')
                    else:
                      st.write(pagent_response)

                    #Generate a Natural Language Answer
                    if 'sql_query' in st.session_state and 'result' in st.session_state:
                      answer = rephrase_answer.invoke({"question": question, "query": st.session_state['sql_query'], "result": st.session_state['result']})
                      return answer

            answer = generate_and_execute_query(question)

            #Display Natural Language Answer
            st.write("Result in Natural Language:")
            st.write(answer)

            exit_app = st.button("Exit App")
            if exit_app:
                #Disconnecti Database and Exit
                st.session_state["db_connected"] = False
                st.success("The database connection has been closed.")
                st.stop()
                st.rerun()
    else:
        st.warning("Please upload and connect to a database before asking a question.")

UserInputQuery()

Overwriting QueryData_AI.py


In [ ]:
from pyngrok import ngrok
!ngrok config add-authtoken #ENTER YOUR AUTH TOKEN HERE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &>/content/logs.txt &
public_url = ngrok.connect(addr="8501", proto="http")
public_url

<NgrokTunnel: "https://9961-35-238-228-170.ngrok-free.app" -> "http://localhost:8501">